In [5]:
%pip install --target=$nb_ath unidecode

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 65.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 9.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 7.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 17.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 5.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:0000:0100:01
     

In [6]:
import io
from unidecode import unidecode
import re
import os 
import csv 
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
maxlen1,maxlen2=10,10

In [7]:
def sentence2wordlist(sentence,putEOS=1):
    words=unidecode(sentence).lower().strip()
    words=re.sub(r"([',.!?\d])", r"\1", words)
    words=re.sub(r"[^a-zA-Z,.!?\d]+", r" ", words)
    ret=words.split()
    if putEOS:
        ret.append('<EOS>')
    return ret
sentence2wordlist("[ab c'deFg hIj.7k+ -*n",putEOS=1)



['ab', 'c', 'defg', 'hij.7k', 'n', '<EOS>']

In [9]:
def create_files(maxlen1=maxlen1, maxlen2=maxlen2, masterpath='/Users/JUDE.RAY.2023/Pytorch-Seq2seq-translator', srcfile='eng-fra.txt',\
                    datafiles=('eng-fra_train.txt', 'eng-fra-test.txt'), vocafiles=('engLookup.csv', 'fraLookup.csv'), ratio=(9,1), maxlines=int(1e6)):
    linecount = 0
    traincount = 0
    testcount = 0
    pool=ratio[0] + ratio[1]
    tests=[]
    wordbank1=['<PAD>', '<SOS>', '<EOS']
    wordbank2=['<PAD>', '<SOS>', '<EOS>']
    with open(masterpath+datafiles[0], "w") as f1:
        with open((masterpath+srcfile), encoding="utf-8") as f:
            for line in f:
                randvar = torch.randint(pool,(1,))
                if randvar < ratio[0]:
                    s1,s2=line.split('\t')
                    words1 = sentence2wordlist(s1)
                    words2 = sentence2wordlist(s2)
                    if len(words1) <= maxlen1 and len(words2) <= maxlen2:
                        f1.write(line)
                        for word in words1:
                            if not word in wordbank1: wordbank1.append(word)
                        for word in words2:
                            if not word in wordbank2: wordbank2.append(word)
                        traincount += 1
                    elif randvar >= ratio[0] and randvar < ratio[1] + ratio[0]:
                       tests.append(line)
                    if linecount > maxlines:
                        break
                    linecount += 1
                with open(masterpath + datafiles[1], "w") as f2:
                    for line in tests:
                        s1, s2 = line.split('\t')
                        words1 = sentence2wordlist(s1)
                        words2 = sentence2wordlist(s2)
                        if len(words1) <= maxlen1 and len(words2) <= maxlen2:
                            invocab = 1
                            for word1 in words1:
                                if not word1 in wordbank1:
                                    invocab = 0
                                    break
                            for word2 in words2:
                                if not word2 in wordbank2:
                                    invocab=0
                                    break
                            if invocab:
                                f2.write(line)
                                testcount += 1
                        for savecount,wordbank in enumerate((wordbank1,wordbank2)):
                            mydict = dict(zip(list(range(0,len(wordbank))), wordbank))
                            with open(masterpath + vocafiles[savecount], "w", newline="") as fp:
                                writer = csv.DictWriter(fp, fieldnames=mydict.keys())
                                writer.writeheader()
                                writer.writerow(mydict)
                            return linecount, (traincount, testcount)
                        create_files(ratio=(98,2))

